<a href="https://colab.research.google.com/github/il-piccolo-machiavelli/ai-class/blob/main/%EC%96%B8%EC%96%B4%EB%AA%A8%EB%8D%B8%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%8B%AC%ED%94%8C%ED%95%9C_%EB%B9%88%EC%B9%B8_%EC%B1%84%EC%9A%B0%EA%B8%B0_%EA%B2%8C%EC%9E%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

원본 게임 전체 코드

In [ ]:
!pip install torch
!pip install transformers

import random
import time
from transformers import pipeline

a = [""]
answer = ""
total_score = 0
total_question = 1
correct_answers = 0

gen = pipeline(
    'text-generation',
    model='bigscience/bloom-560m',
    max_length=50,
    temperature=0.7,
    do_sample=True,
    truncation=True,
    clean_up_tokenization_spaces=True
)

def parser(myText):
  tokens = []
  word = ""
  for char in myText:
    if char.isalnum():
      word += char
    else:
      if word:
        tokens.append(word)
        word = ""
      tokens.append(char)
  if word:
    tokens.append(word)
  return tokens

def wordlist(myList):
  return parser(random.choice(myList))

def blank(myList):
  tokens = wordlist(myList)
  words = []
  for word in tokens:
    if len(word) >= 3:
      words.append(word)
  if len(words) == 0:
    return None
  selected = random.choice(words)
  masked = "_" * len(selected)
  result = []
  for word in tokens:
    if word == selected:
      result.append(masked)
    else:
      result.append(word)
  return "".join(result), selected

def get_answer(prompt):
  reply = gen(prompt,num_return_sequences=1)[0]['generated_text']
  texts = reply.replace("\n"," ").strip()
  text = texts[len(prompt):].strip()
  for delimiter in ['.','!','?','\'','"']:
    if delimiter in text:
      text = text.split(delimiter)[0] + delimiter
      break
  while text and not text[0].isalpha():
        text = text[1:].strip()
  if len(text.split()) < 4:
    return get_answer(prompt)
  return text

preset = get_answer("You are my friend. Please speak in English.")

print("FILL IN THE BLANK BELOW")
print("-"*80)

while answer != "/q":
  print("Generating a new question...\n\n")

  a[0] = get_answer("Give me one simple random quote of people.")
  b = blank(a)
  attempt = 0
  question_score = 0
  start_time = 0
  elapsed_time = 0
  if b == None:
    continue
  else:
    print(f"{total_question}. \"{b[0]}\"\n")
    while answer != b[1]:
      answer = input("Input your answer (exit:/q):")
      start_time = time.time()
      if answer == "/q":
        break
      elif answer == b[1]:
        print("Correct!")
        question_score = 3 - attempt
        elapsed_time = time.time() - start_time
        if elapsed_time <= 5:
          question_score += 2
          print("You've got 2 time bonus points.")
        correct_answers += 1
      elif answer != b[1]:
        hint = ""
        if attempt == 0:
          hint = "*" * (len(b[1])-1)
          hint = b[1][0] + hint
        elif attempt == 1:
          hint = "*" * (len(b[1])-2)
          hint = b[1][0] + hint + b[1][-1]
        else:
          print(f"You've used all attempts. The answer was \'{b[1]}\'.")
          break
        print(f"Incorrect! (hint:{hint})")
      attempt += 1
    if answer != "/q":
      accuracy = (correct_answers/total_question)*100
      total_score += question_score
      total_question += 1
      print(f"You've got total {question_score} points in this round.")
      print(f"Your total points are {total_score}.")
      print(f"Accuracy: {accuracy:.2f}%\n")
print("\nThank you for playing!")

Flask 사용 게임 코드

In [3]:
!ls templates 2>/dev/null || mkdir templates

In [ ]:
%%writefile templates/game.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Fill in the Blank Game</title>
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: #f0f2f5;
            margin: 0;
            padding: 20px;
            display: flex;
            justify-content: center;
            min-height: 100vh;
        }

        .container {
            max-width: 800px;
            width: 100%;
        }

        .game-header {
            background-color: #ffffff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            margin-bottom: 20px;
        }

        .game-title {
            color: #1a73e8;
            margin: 0;
            font-size: 24px;
            margin-bottom: 10px;
        }

        .score-board {
            display: flex;
            gap: 20px;
            margin-top: 10px;
            padding: 10px;
            background-color: #f8f9fa;
            border-radius: 5px;
        }

        .score-item {
            flex: 1;
            text-align: center;
        }

        .score-label {
            font-size: 14px;
            color: #5f6368;
        }

        .score-value {
            font-size: 20px;
            color: #1a73e8;
            font-weight: bold;
        }

        .question-container {
            background-color: #ffffff;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            margin-bottom: 20px;
        }

        .question-number {
            color: #5f6368;
            font-size: 16px;
            margin-bottom: 15px;
        }

        .question-text {
            font-size: 20px;
            color: #202124;
            margin-bottom: 25px;
            line-height: 1.5;
        }

        .timer {
            font-size: 14px;
            color: #5f6368;
            text-align: right;
            margin-bottom: 10px;
        }

        .input-group {
            display: flex;
            gap: 10px;
            margin-bottom: 20px;
        }

        .answer-input {
            flex: 1;
            padding: 12px;
            font-size: 16px;
            border: 2px solid #e8eaed;
            border-radius: 5px;
        }

        .answer-input:focus {
            outline: none;
            border-color: #1a73e8;
        }

        .submit-button {
            background-color: #1a73e8;
            color: white;
            border: none;
            padding: 0 25px;
            border-radius: 5px;
            cursor: pointer;
            font-size: 16px;
            transition: background-color 0.2s;
        }

        .submit-button:hover {
            background-color: #1557b0;
        }

        .hint {
            color: #5f6368;
            font-size: 14px;
            margin-top: 10px;
        }

        .loading {
            background-color: #e6f4ea;
            color: #137333;
            padding: 15px;
            border-radius: 5px;
            margin-top: 15px;
            display: flex;
            align-items: center;
            gap: 12px;
            display: none;
        }

        @keyframes spin {
            to { transform: rotate(360deg); }
        }

        .spinner {
            width: 20px;
            height: 20px;
            border: 3px solid #e6f4ea;
            border-top: 3px solid #137333;
            border-radius: 50%;
            animation: spin 1s linear infinite;
        }

        .feedback {
            padding: 15px;
            border-radius: 5px;
            margin-top: 15px;
            display: none;
        }

        .feedback.correct {
            background-color: #e6f4ea;
            color: #137333;
        }

        .feedback.incorrect {
            background-color: #fce8e6;
            color: #c5221f;
        }

        .bonus {
            background-color: #e8f0fe;
            color: #1a73e8;
            padding: 10px;
            border-radius: 5px;
            margin-top: 10px;
            display: none;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="game-header">
            <h1 class="game-title">Fill in the Blank</h1>
            <div class="score-board">
                <div class="score-item">
                    <div class="score-label">Total Score</div>
                    <div class="score-value" id="total-score">0</div>
                </div>
                <div class="score-item">
                    <div class="score-label">Accuracy</div>
                    <div class="score-value" id="accuracy">0%</div>
                </div>
                <div class="score-item">
                    <div class="score-label">Questions Solved</div>
                    <div class="score-value" id="questions-solved">0</div>
                </div>
            </div>
        </div>

        <div class="question-container">
            <div class="question-number" id="question-number">Question #1</div>
            <div class="question-text" id="question-text"></div>
            <div class="timer" id="timer">Time bonus remaining: 5s</div>
            <div class="input-group">
                <input type="text" class="answer-input" id="answer-input" placeholder="Type your answer here" autocomplete="off">
                <button class="submit-button" id="submit-button">Submit</button>
            </div>
            <div class="hint" id="hint"></div>
            <div class="loading" id="loading">
                <div class="spinner"></div>
                <span>Generating a new question...</span>
            </div>
            <div class="feedback correct" id="correct-feedback"></div>
            <div class="feedback incorrect" id="incorrect-feedback"></div>
            <div class="bonus" id="bonus-message"></div>
        </div>
    </div>

    <script>
        const sessionId = "{{ session_id }}";
        let timerInterval;
        let timeLeft = 5;

        function startTimer() {
            timeLeft = 5;
            clearInterval(timerInterval);
            timerInterval = setInterval(() => {
                timeLeft--;
                if (timeLeft >= 0) {
                    $('#timer').text(`Time bonus remaining: ${timeLeft}s`);
                } else {
                    clearInterval(timerInterval);
                }
            }, 1000);
        }

        function generateNewQuestion() {
            $('#loading').show();
            $('.feedback, .bonus').hide();
            $('#answer-input').prop('disabled', true);
            $('#submit-button').prop('disabled', true);

            $.ajax({
                url: '/generate_question',
                method: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ session_id: sessionId }),
                success: function(response) {
                    if (response.status === 'success') {
                        $('#question-text').html(response.question);
                        $('#question-number').text(`Question #${response.question_number}`);
                        $('#answer-input').val('').prop('disabled', false).focus();
                        $('#submit-button').prop('disabled', false);
                        $('#hint').text('');
                        startTimer();
                    }
                    $('#loading').hide();
                }
            });
        }

        function submitAnswer() {
            const answer = $('#answer-input').val().trim();
            if (!answer) return;

            $.ajax({
                url: '/check_answer',
                method: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({
                    session_id: sessionId,
                    answer: answer
                }),
                success: function(response) {
                    $('.feedback, .bonus').hide();
                    if (response.status === 'correct') {
                        $('#correct-feedback').text('Correct!').show();
                        $('#total-score').text(response.total_score);
                        $('#accuracy').text(response.accuracy + '%');
                        $('#questions-solved').text(parseInt($('#questions-solved').text()) + 1);

                        if (response.time_bonus) {
                            $('#bonus-message').text("You've got 2 time bonus points!").show();
                        }

                        setTimeout(generateNewQuestion, 2000);
                    } else {
                        if (response.all_attempts_used) {
                            $('#incorrect-feedback').text(`You've used all attempts. The answer was '${response.correct_answer}'!`).show();
                            setTimeout(generateNewQuestion, 2000);
                        } else {
                            $('#incorrect-feedback').text(`Incorrect! (hint: ${response.hint})`).show();
                            $('#answer-input').val('').focus();
                        }
                    }
                }
            });
        }

        $(document).ready(function() {
            generateNewQuestion();

            $('#submit-button').click(submitAnswer);

            $('#answer-input').keypress(function(e) {
                if (e.which == 13) {
                    submitAnswer();
                }
            });
        });
    </script>
</body>
</html>

In [ ]:
!ls

In [ ]:
!pip install torch
!pip install transformers
!pip install flask

In [ ]:
import random
import time
from transformers import pipeline
from flask import Flask, render_template, request, jsonify
from google.colab import output

app = Flask(__name__)
game_states = {}

gen = pipeline(
    'text-generation',
    model='bigscience/bloom-560m',
    max_length=50,
    temperature=0.7,
    do_sample=True,
    truncation=True,
    clean_up_tokenization_spaces=True
)

def parser(myText):
    tokens = []
    word = ""
    for char in myText:
        if char.isalnum():
            word += char
        else:
            if word:
                tokens.append(word)
                word = ""
            tokens.append(char)
    if word:
        tokens.append(word)
    return tokens

def wordlist(myList):
    return parser(random.choice(myList))

def blank(myList):
    tokens = wordlist(myList)
    words = []
    for word in tokens:
        if len(word) >= 3:
            words.append(word)
    if len(words) == 0:
        return None
    selected = random.choice(words)
    masked = "_" * len(selected)
    result = []
    for word in tokens:
        if word == selected:
            result.append(masked)
        else:
            result.append(word)
    return "".join(result), selected

def get_answer(prompt):
    reply = gen(prompt,num_return_sequences=1)[0]['generated_text']
    texts = reply.replace("\n"," ").strip()
    text = texts[len(prompt):].strip()
    for delimiter in ['.','!','?',"'",'"']:
        if delimiter in text:
            text = text.split(delimiter)[0] + delimiter
            break
    while text and not text[0].isalpha():
        text = text[1:].strip()
    if len(text.split()) < 4:
        return get_answer(prompt)
    return text

@app.route('/')
def home():
    session_id = str(random.randint(1000, 9999))
    game_states[session_id] = {
        'total_score': 0,
        'total_question': 1,
        'correct_answers': 0,
        'attempt': 0,
        'current_answer': '',
        'question_score': 0
    }
    return render_template('game.html', session_id=session_id)

@app.route('/generate_question', methods=['POST'])
def generate_question():
    session_id = request.json['session_id']
    state = game_states[session_id]

    a = [""]
    a[0] = get_answer("Give me one simple random quote of people.")
    b = blank(a)

    if b is None:
        return jsonify({'status': 'error', 'message': 'Failed to generate question'})

    state['current_question'] = b[0]
    state['current_answer'] = b[1]
    state['attempt'] = 0
    state['question_score'] = 0
    state['start_time'] = time.time()

    return jsonify({
        'status': 'success',
        'question': b[0],
        'question_number': state['total_question']
    })

@app.route('/check_answer', methods=['POST'])
def check_answer():
    data = request.json
    session_id = data['session_id']
    user_answer = data['answer']
    state = game_states[session_id]

    if user_answer == state['current_answer']:

        state['question_score'] = 3 - state['attempt']
        elapsed_time = time.time() - state['start_time']
        time_bonus = False

        if elapsed_time <= 5:
            state['question_score'] += 2
            time_bonus = True

        state['correct_answers'] += 1
        state['total_score'] += state['question_score']
        accuracy = (state['correct_answers'] / state['total_question']) * 100

        response = {
            'status': 'correct',
            'score': state['question_score'],
            'total_score': state['total_score'],
            'accuracy': round(accuracy, 2),
            'time_bonus': time_bonus
        }

        state['total_question'] += 1

    else:
        hint = ""
        if state['attempt'] == 0:
            hint = "*" * (len(state['current_answer'])-1)
            hint = state['current_answer'][0] + hint
        elif state['attempt'] == 1:
            hint = "*" * (len(state['current_answer'])-2)
            hint = state['current_answer'][0] + hint + state['current_answer'][-1]

        response = {
            'status': 'incorrect',
            'hint': hint if state['attempt'] < 2 else None,
            'all_attempts_used': state['attempt'] >= 2,
            'correct_answer': state['current_answer'] if state['attempt'] >= 2 else None
        }

        state['attempt'] += 1

    return jsonify(response)

if __name__ == '__main__':
    output.serve_kernel_port_as_iframe(5000, height=700)
    app.run()